In [7]:
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [8]:
class ChatBot:
    def __init__(self):

        link="microsoft/DialoGPT-large"
        self.chat_history_ids = None
        self.bot_input_ids = None
        self.end_chat = False
        self.tokenizer = AutoTokenizer.from_pretrained(link)
        self.model = AutoModelForCausalLM.from_pretrained(link)
        self.greet_user()

    def greet_user(self):

        print("ChatBot : Welcome, I am a ChatBot, ready to assist you.\n Type 'bye', 'quit', or 'exit' to end the chat\n")

    def user_input(self):
        text = input("User    : ")
        if text.lower().strip() in ['bye', 'quit', 'exit']:
            self.end_chat = True
            print('Quitting ChatBot')
        else:
            self.new_user_input_ids = self.tokenizer.encode(text + self.tokenizer.eos_token, return_tensors='pt')

    def generate_response(self):
        if self.chat_history_ids is not None:
            self.bot_input_ids = torch.cat([self.chat_history_ids, self.new_user_input_ids], dim=-1)
        else:
            self.bot_input_ids = self.new_user_input_ids

        self.chat_history_ids = self.model.generate(
            self.bot_input_ids, max_length=1000, pad_token_id=self.tokenizer.eos_token_id
        )

        response = self.tokenizer.decode(
            self.chat_history_ids[:, self.bot_input_ids.shape[-1]:][0], skip_special_tokens=True
        )

        if not response:
            response = self.get_fallback_response()

        print('ChatBot :', response)

    def get_fallback_response(self):
        i = -1
        response = self.tokenizer.decode(
            self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], skip_special_tokens=True
        )

        while not response:
            i -= 1
            response = self.tokenizer.decode(
                self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], skip_special_tokens=True
            )


        if response.strip() == '?':
            return "I don't know"
        else:
            return "Okay"

In [ ]:
bot = ChatBot()

while True:
    bot.user_input()
    if bot.end_chat:
        break
    bot.generate_response()

ChatBot : Welcome, I am a ChatBot, ready to assist you.
 Type 'bye', 'quit', or 'exit' to end the chat

User    : Hey
ChatBot : Hey, how are you?
User    : I'm good. How are you?
ChatBot : I'm good.
User    : Want to help on coding?
ChatBot : I can help.
User    : Do you code?
ChatBot : I can code.
User    : Which language do you code in?
ChatBot : C, but I'm not a programmer.
User    : You tried using C++?
ChatBot : I can code in C.
User    : What is the best language to code in?
ChatBot : C is the best language to code in.
User    : Do you like tea?
ChatBot : I like tea.
User    : Which type of tea do you prefer?
ChatBot : C is the best.
